##데이터 전처리

###bond_report

In [ ]:
pip install ekonlpy

In [ ]:
#라이브러리
import re
import time
import datetime
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [ ]:
import pandas as pd
bond_df=pd.read_csv("./Bond_report_naver.csv")
bond_report=pd.DataFrame(set([(bond_df.loc[i,"0"], bond_df.loc[i,"1"]) for i in range(len(bond_df))]))    # 중복 행 제거
bond_report.columns=["DATE", "CONTENTS"]

In [ ]:
bond_report["CONTENTS"]=[eval(bond_report.loc[i,"CONTENTS"]) for i in range(len(bond_report))] #str → list
report_content=[]
for doc in bond_report["CONTENTS"]:
    idx=2       #3번째 줄부터 저장
    content=[]
    while idx<len(doc)-1:
        if  doc[idx].startswith("자료 출처:") or doc[idx].startswith("자료:") or doc[idx].startswith("그림") or doc[idx].startswith("도표") or len(doc[idx])<2: #표, 그래프, 자료출처 문장 제외
            idx+=1
        elif doc[idx].startswith("(단위:"): #(단위: ) = 표 내용 → 다음 2문장까지 같이 삭제
            idx+=3
        elif len(re.findall(r"\d+(\.\d+)?", doc[idx]))> len(re.findall("[가-힣a-zA-Z]+", doc[idx])): #숫자가 문자보다 많을경우 문장 삭제
            idx+=1
        elif doc[idx].startswith("[ Compliance Notice ]"):  #뒷내용 삭제
          break
        else:
            content.append(doc[idx])
            idx+=1
    report_content.append(content)

bond_report["CLEANED_CONTENTS"]=report_content

In [ ]:
#텍스트 CLEANED_CONTENTS 한 뭉치로 변환
bond_report["CLEANED_CONTENTS"]=["".join(bond_report.loc[i,"CLEANED_CONTENTS"]) for i in range(len(bond_report))]

#텍스트 정규표현식으로 정제
def clean(df, regex, replace):
    df["CLEANED_CONTENTS"]=df["CLEANED_CONTENTS"].replace(regex,replace, regex=True)

clean(bond_report, r"[^.]*니다 ?[.]", "")      #'니다'로 끝나는 문장 삭제
clean(bond_report, r'\[[^]]*\]', "")           #[] 삭제
clean(bond_report, r'\S+@\S+', "")             #이메일
clean(bond_report, r'.\d+-\d+.*?',"")          #전화번호
clean(bond_report, r'\d+\.\d*',"`")
bond_report

,날짜,내용(원본),내용
0,2023-08-31,"[미국 국채, 투자 지형도 변화, [Bond Strategy] 김명실(2122-92...",미국 국채 시장의 수급적 특징 5가지와 시사점1) 작년 하반기부터 미국 국채의 주요...
1,2023-08-31,"[31 Aug 2023, (%, ), ▪ : 3Y( ), 10Y( ) 8 28 30...",KR MarketKR Bond YTM 1D 1W 1M KTB Futures 1D ...
2,2023-08-31,"[(단위: 억원, %), 발간일: 8.31 (목), 8.30 (수) 1d 5d 1M...","8.30 1d 5d 1M 전일 주요 채권 관련 기사한신평,GS건설'A+부정적'등급..."
3,2023-08-31,"[Eugene’s FICC Update, Fixed Income 김지나_ 02)36...",Fixed Income 외국인 3년 국채선물 수급 외국인 10년 국채선물 수급채권일...
4,2023-08-31,"[해외크레딧 2023년 8월 31일, 美 FDIC의 대형은행 부채 규제, 강화안, ...",강화안Credit Analyst 금융 안정과 예금 보호를 위한 은행 규제 강화안 발...
...,...,...,...
5608,2008-04-21,"[항상 열려는 있는 가능성 2008. 4. 21, Fixed Income Weekl...",경기 전망은 비교적 분명… 인플레 명분이 시점 결정할 것현 금리 레벨이 5월 인하 ...
5609,2008-04-14,"[총재의 변신은 무죄다?! 2008. 4. 14, Fixed Income Weekl...","4월 금통위는 정부-한은간 policy mix 합의의 반영일 수도예상대로, 올해 한..."
5610,2008-04-07,"[Fixed Income Weekly, 2008. 4. 7. #918, Fixed ...","Fixed Income Weekly4월 7일돌아온 박스권대우증권 리서치센터다만, 하..."
5611,2008-04-07,"[Fixed Income Weekly, 2008. 4. 7. #918, Fixed ...","Fixed Income Weekly4월 7일돌아온 박스권대우증권 리서치센터다만, 하..."


In [ ]:
analysts=["안재균", "정혜진", "김상만", "김상인", "김지나", "이정훈", "박민영", "김상훈", "김현지", "권재형", "구혜영", "김민형", "이경록", "박승진", "김정형", "안예하", "박진영", "안주영", "전병하", "오창섭", "윤일광", "서철수", "윤여삼","유욱재","모승규"]
for name in analysts:
  bond_report["CLEANED_CONTENTS"]=bond_report["CLEANED_CONTENTS"].replace(name,"", regex=True)

In [ ]:
#내용확인
bond_report.loc[3541,"CLEANED_CONTENTS"]

In [ ]:
from ekonlpy import Mecab
with open("./stopword.txt", "r") as f:
  stopword=f.read().splitlines()

mecab=Mecab()
pos_list=[mecab.replace_synonyms(bond_report.loc[row, "CLEANED_CONTENTS"]) for row in range(len(bond_report))]    #tokenize
pos_tag=[[token for token in row if token[1] in ['NNG', "VA", "VAX", "MAG", "V"] and token[0] not in stopword] for row in pos_list]   #stopword 제외, 특정 품사만 저장
bond_report["TOKEN"]=pos_tag

In [ ]:
bond_report["TOKEN"]=bond_report["TOKEN"].map(lambda x: None if x == [] else x)
bond_report.dropna(inplace=True)

###kyobo_report

In [ ]:
kyobo_df=pd.read_csv("./Bond_report_kyobo.csv")
kyobo_report=pd.DataFrame(set([(kyobo_df.loc[i,"0"], kyobo_df.loc[i,"1"]) for i in range(len(kyobo_df))]))    #중복 행 제거
kyobo_report.columns=["DATE", "CONTENTS"]

In [ ]:
kyobo_report["CONTENTS"]=[eval(kyobo_report.loc[i,"CONTENTS"]) for i in range(len(kyobo_report))]
report_content=[]
for doc in kyobo_report["CONTENTS"]:
    idx=0
    content=[]
    while idx<len(doc)-1:
        if idx==0:
            if doc[0].startswith("Daily") or doc[0].startswith("Fixed") or doc[0].startswith("발간일자"):
                idx+=3
            elif doc[0].startswith("채권전략 연구위원"):
                idx+=6
            else:
                content.append(doc[0])
                idx+=1

        if doc[idx].startswith("국내주요금리 (%, bp)"):  #표 내용 제외
          break
        elif doc[idx].startswith("자료:") or doc[idx].startswith("[도표") or doc[idx].startswith("주:") or doc[idx].startswith("도표") or len(doc[idx])<2: #표, 그래프, 자료출처 문장 제외
            idx+=1
        elif len(re.findall(r"\d+(\.\d+)?", doc[idx]))> len(re.findall("[가-힣a-zA-Z]+", doc[idx])):
            idx+=1
        else:
            content.append(doc[idx])
            idx+=1
    report_content.append(content)

kyobo_report["CLEANED_CONTENTS"]=report_content

In [ ]:
#텍스트 내용 한 뭉치로 변환
kyobo_report["CLEANED_CONTENTS"]=["".join(kyobo_report.loc[i,"CLEANED_CONTENTS"]) for i in range(len(kyobo_report))]

#텍스트 정규표현식으로 정제
def clean(df, regex, replace):
    df["CLEANED_CONTENTS"]=df["CLEANED_CONTENTS"].replace(regex,replace, regex=True)

clean(kyobo_report, r"[^.]*니다 ?[.]", "")      #'니다'로 끝나는 문장 삭제
clean(kyobo_report, r'\[[^]]*\]', "")           #[] 삭제
clean(kyobo_report, r'\S+@\S+', "")             #이메일
clean(kyobo_report, r'.\d+-\d+.*?',"")          #전화번호
clean(kyobo_report, r'\d+\.\d*',"`")
clean(kyobo_report, r'■', ".")
clean(kyobo_report, r'▪', ".")

#이름 삭제
analysts=['백윤민','권한욱','이슬비','정윤정','공동락', '이정준']
for name in analysts:
  kyobo_report["CLEANED_CONTENTS"]=kyobo_report["CLEANED_CONTENTS"].replace(name,"", regex=True)

In [ ]:
#내용확인
kyobo_report.loc[19,"CLEANED_CONTENTS"]

'상하 달ㅁ채권 금리, 시장안정화 조치로 다소 진정 러 인덱스, 소폭 하락■ 전일 동향- 전일 국내 채권시장에서 국고채 금리는 한은의 통안채 발행 축소 및 정부의105 달러인덱스국고채 발행 축소 등 시장안정화 조치 영향으로 하락 마감. 국채선물 시장에서외국인은 4거래일만에단 기물을 순매수, 장기물은 순매도 전환 100- 해외: 미국 채권시장은 트럼프 당선인의 정책, 이탈리아 국민투표 등 대내외불확실성 요인들이 지속됐지만, 최근 금리 급등세에 따른 저가매수 수요 둥으로 90강세 마감 85■ 금일 전망- 금일 채권시장은 글로벌 금리 급등세가 진정되는 모습을 보이는 가운데, 한은과 정부의 시장 안정화 조치 영향 등으로 추가적인 금리 하락이 예상. 그러나 이탈 70리아 국민투표, 12월 FOMC 등 대외 이벤트들이 예정되어 있기 때문에 시장에 대 Jan-10 Jan-11 Jan-12 Jan-13 Jan-14 Jan-15 Jan-16한 경계 심리는 지속될 것으로 판단'

In [ ]:
from ekonlpy import Mecab
with open("./stopword.txt", "r") as f:
  stopword=f.read().splitlines()

mecab=Mecab()
pos_list=[mecab.replace_synonyms(kyobo_report.loc[row, "CLEANED_CONTENTS"]) for row in range(len(kyobo_report))]
pos_tag=[[token for token in row if token[1] in ['NNG', "VA", "VAX", "MAG", "V"] and token[0] not in stopword] for row in pos_list]
kyobo_report["TOKEN"]=pos_tag

In [ ]:
from collections import Counter
Counter(pos_tag)